## Load YOLO

In [ ]:
from ultralytics import YOLO
# Loading trained YOLO model
model = YOLO("/data_vault/hexai/OAICartilage/runs/detect/SagittalKneeJoint/weights/best.pt")

# Run prediction and save cropped images
results = model.predict("/data_vault/hexai/OAICartilage/Image", save=True, save_crop=True)

## Extracting Bounding Box Coordinates from YOLO Predictions

In [ ]:
# Extract Bounding Boxes and Save
import os
import re
bbox_dict = {}  
for result in results:
    image_name = os.path.basename(result.path)  # Get image filename
    
    # ID + slice number as string
    match = re.match(r"(\d+_\d+_\d+)_slice_(\d+)", image_name)
    if not match:
        print(f"Unrecognized filename format: {image_name}")
        continue  # Skip unrecognized filenames

    study_id, slice_num_str = match.groups() 

    boxes = result.boxes.xyxy.cpu().numpy()  # Extract bounding box coordinates (x1, y1, x2, y2)

    # Print progress for each image processed
    print(f"Processing {image_name} (ID: {study_id}, Slice: {slice_num_str})")

    # Store bounding boxes in bbox_dict, keeping slice_num_str as string
    if study_id not in bbox_dict:
        bbox_dict[study_id] = {}
    
    # Store boxes under study ID -> slice number as string
    bbox_dict[study_id][slice_num_str] = boxes

    # Print the bounding boxes for this image
    if len(boxes) == 0:
        print("  No detections")
    else:
        for idx, box in enumerate(boxes):
            print(f" Bounding box {idx+1}: {box}")

In [ ]:
# Save bounding boxes to a .pkl file
bbox_save_path = "/data_vault/hexai/OAICartilage/bbox_dict.pkl"
with open(bbox_save_path, "wb") as f:
    pickle.dump(bbox_dict, f)

print(f"Bounding boxes saved successfully to {bbox_save_path}")

## Crop Images based on Bbox

In [ ]:
# Crop image based on bbox
import os
import pickle
from PIL import Image

# Paths
image_folder = "/data_vault/hexai/OAICartilage/Image"  # Original images
bbox_pickle_path = "/data_vault/hexai/OAICartilage/bbox_dict.pkl"
output_crop_folder = "/data_vault/hexai/OAICartilage/image_manual_crops"
os.makedirs(output_crop_folder, exist_ok=True)

# Manual cropping
for study_id, slices in bbox_dict.items():
    for slice_num_str, bboxes in slices.items():
        image_filename = f"{study_id}_slice_{slice_num_str}.jpg"
        image_path = os.path.join(image_folder, image_filename)

        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        image = Image.open(image_path)

        for idx, (x1, y1, x2, y2) in enumerate(bboxes):
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            cropped_image = image.crop((x1, y1, x2, y2))

            crop_filename = f"{study_id}_slice_{slice_num_str}.jpg"
            crop_path = os.path.join(output_crop_folder, crop_filename)
            cropped_image.save(crop_path)

print("✅ Manual cropping complete using bounding box data.")

In [ ]:
# Reloading bbox
import pickle

# Load bbox_dict from a pickle file
with open("/data_vault/hexai/OAICartilage/bbox_dict.pkl", "rb") as f:
    bbox_dict = pickle.load(f)

print("✅ Loaded bbox_dict successfully!")


## SIMPLE ITK LIBRARY 

In [ ]:
import os
import SimpleITK as sitk
import numpy as np

# Paths
annotation_folder = "/data_vault/hexai/OAICartilage/Annotations copy"
output_folder = "/data_vault/hexai/OAICartilage/cropped_annotations_simpleitk_v2"
os.makedirs(output_folder, exist_ok=True)


# Process each annotation file 
for study_id, slices in bbox_dict.items():
    input_path = os.path.join(annotation_folder, f"{study_id}.nii.gz")
    if not os.path.exists(input_path):
        print(f"Annotation not found: {input_path}")
        continue

    # Read ITK-SNAP NIfTI image 
    img = sitk.ReadImage(input_path)
    img_np = sitk.GetArrayFromImage(img)  # Shape

    # Store spatial metadata
    spacing = img.GetSpacing()
    origin = img.GetOrigin()
    direction = img.GetDirection()

    print(f"Processing {study_id} | Volume shape: {img_np.shape}")

    study_output_dir = os.path.join(output_folder, study_id)
    os.makedirs(study_output_dir, exist_ok=True)

    # Loop through annotated slices
    for slice_num_str, bboxes in slices.items():
        slice_index = int(slice_num_str) - 1  # YOLO slices are 1-based

        if slice_index < 0 or slice_index >= img_np.shape[0]:
            print(f"Skipping slice {slice_num_str} (out of bounds)")
            continue

        slice_2d = img_np[slice_index, :, :]  # [Y, X]

        for idx, (x1, y1, x2, y2) in enumerate(bboxes):
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Ensure bounding box is valid
            if (
                x1 < 0 or y1 < 0 or
                x2 > slice_2d.shape[1] or y2 > slice_2d.shape[0]
            ):
                print(f"Skipping invalid crop for {study_id} slice {slice_num_str}")
                continue

            # Crop annotation from slice
            cropped_np = slice_2d[y1:y2, x1:x2]
            cropped_3d = np.expand_dims(cropped_np, axis=0)  # [1, Y, X]

            cropped_img = sitk.GetImageFromArray(cropped_3d)
            cropped_img.SetSpacing(spacing)
            cropped_img.SetOrigin(origin)
            cropped_img.SetDirection(direction)

            output_path = os.path.join(
                study_output_dir,
                f"{study_id}_slice_{int(slice_num_str):03d}_crop{idx}.nii.gz"
            )
            sitk.WriteImage(cropped_img, output_path)
            print(f"Saved: {output_path}")
            # Save NumPy version
            
            numpy_output_dir = "/data_vault/hexai/OAICartilage/cropped_annotations_numpy"
            os.makedirs(os.path.join(numpy_output_dir, study_id), exist_ok=True)
            
            np_save_path = os.path.join(
                numpy_output_dir, study_id,
                f"{study_id}_slice_{int(slice_num_str):03d}.npy"
            )
            np.save(np_save_path, cropped_np)
            print(f"Saved NumPy: {np_save_path}")

## Dimension Check

In [ ]:
from PIL import Image
import nibabel as nib
import numpy as np

# Load any YOLO-cropped image
cropped_img_path = "/data_vault/hexai/OAICartilage/image_manual_crops/9036771_20060321_10899612_slice_114.jpg"  
cropped_img = Image.open(cropped_img_path)
print(f"Cropped image dimensions: {cropped_img.size}")


cropped_annotation = "/data_vault/hexai/OAICartilage/cropped_annotations/9036771_20060321_10899612/9036771_20060321_10899612_slice_114.nii.gz"

# Check if the file exists
try:
    # Load the NIfTI slice
    cropped_annotation = nib.load(cropped_annotation)
    cropped_annotation_slice_data = np.squeeze(cropped_annotation.get_fdata())  

    # Print the dimensions of the annotation slice
    print(f"✅ Cropped annotation slice dimensions: {cropped_annotation_slice_data.shape}")
except FileNotFoundError:
    print(f"❌  slice not found: {cropped_annotation_slice_data}")


## VISUALIZATION

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

nifti_path = "/data_vault/hexai/OAICartilage/cropped_annotations/9021195_20050531_10534714/9021195_20050531_10534714_slice_036.nii.gz"


image = sitk.ReadImage(nifti_path)
image_np = sitk.GetArrayFromImage(image)  # Expected shape: (1, height, width)

print(f"NIfTI shape: {image_np.shape}")


if image_np.shape[0] == 1:
    image_np = image_np[0, :, :]


plt.figure(figsize=(6, 6))
plt.imshow(image_np, cmap="gray")
plt.title("Cropped Annotation Slice")
plt.axis("off")
plt.show()

In [ ]:
import os
import numpy as np
import SimpleITK as sitk
from PIL import Image
import matplotlib.pyplot as plt

study_id = "9283264_20040722_10282013"
slice_num = "060"

cropped_image_path = f"/data_vault/hexai/OAICartilage/image_manual_crops/{study_id}_slice_{slice_num}.jpg"
cropped_annotation_path = f"/data_vault/hexai/OAICartilage/cropped_annotations/{study_id}/{study_id}_slice_{slice_num}.nii.gz"

# Load cropped image
img = Image.open(cropped_image_path).convert("RGB")
img_np = np.array(img)

# Load annotation and squeeze to 2D
annotation = sitk.ReadImage(cropped_annotation_path)
annotation_np = sitk.GetArrayFromImage(annotation)

if annotation_np.ndim == 3 and annotation_np.shape[0] == 1:
    annotation_np = annotation_np[0, :, :]  # Convert to 2D


label_colors = {
    1: [0, 255, 0],      
    2: [0, 0, 255],      
    3: [255, 165, 0],    
    4: [255, 0, 0],      
}

overlay = img_np.copy()

for label, color in label_colors.items():
    mask = annotation_np == label
    for c in range(3):  # RGB channels
        overlay[..., c][mask] = (0.6 * img_np[..., c][mask] + 0.4 * color[c]).astype(np.uint8)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img_np)
plt.title("Original Cropped Image")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(annotation_np, cmap="gray")
plt.title("Annotation Mask")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(overlay)
plt.title("Overlay")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import nibabel as nib
from PIL import Image
import matplotlib.pyplot as plt


study_id = "9283264_20040722_10282013"
slice_num = "060"
crop_idx = 0


cropped_img_path = f"/data_vault/hexai/OAICartilage/image_manual_crops/{study_id}_slice_{slice_num}.jpg"
cropped_annotation_path = f"/data_vault/hexai/OAICartilage/cropped_annotations/{study_id}/{study_id}_slice_{slice_num}.nii.gz"

if not os.path.exists(cropped_img_path):
    raise FileNotFoundError(f"Image not found: {cropped_img_path}")

img = Image.open(cropped_img_path).convert("L")
img_np = np.array(img)
print(f"✅ YOLO-cropped image shape (H×W): {img_np.shape}")


if not os.path.exists(cropped_annotation_path):
    raise FileNotFoundError(f"Annotation not found: {cropped_annotation_path}")

annotation_nii = nib.load(cropped_annotation_path)
annotation_data = np.squeeze(annotation_nii.get_fdata())
print(f"✅ Cropped annotation shape (H×W): {annotation_data.shape}")


if annotation_data.shape != img_np.shape:
    print("Shape mismatch detected. Attempting transpose...")
    annotation_data = np.transpose(annotation_data, (1, 0))
    print(f"🔁 Transposed annotation shape: {annotation_data.shape}")

assert annotation_data.shape == img_np.shape, "❌ Shapes still don't match after transpose!"


overlay = np.zeros((img_np.shape[0], img_np.shape[1], 3), dtype=np.uint8)
overlay[..., 0] = img_np  
overlay[..., 1] = (annotation_data * 50).clip(0, 255).astype(np.uint8) 


plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(img_np, cmap="gray")
plt.title("Cropped Image")

plt.subplot(1, 3, 2)
plt.imshow(annotation_data, cmap="gray")
plt.title("Cropped Annotation")

plt.subplot(1, 3, 3)
plt.imshow(overlay)
plt.title("Overlay (Image + Mask)")

plt.tight_layout()
plt.show()

## Renaming Cropped Annotations to match images (If needed)

In [ ]:
# Rename Cropped Files
cropped_annotations_folder = "/data_vault/hexai/OAICartilage/cropped_annotations_simpleitk_v2"

# Iterate through each study ID folder
for study_id in os.listdir(cropped_annotations_folder):
    study_folder_path = os.path.join(cropped_annotations_folder, study_id)
    
    # Ensure it's a directory
    if not os.path.isdir(study_folder_path):
        continue

    # Iterate through files in the study folder
    for filename in os.listdir(study_folder_path):
        if filename.endswith(".nii.gz") and "_crop0" in filename:
            old_path = os.path.join(study_folder_path, filename)
            new_filename = filename.replace("_crop0", "")  # Remove "_crop0"
            new_path = os.path.join(study_folder_path, new_filename)
            
            # Rename the file
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} -> {new_path}")